## RAG System Using Llama2 With Hugging Face

In [ ]:
# !unzip data.zip -d data

Archive:  data.zip
replace data/data/diet_plan/Asthama/000360398.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
%pip install pypdf
%pip install -q transformers einops accelerate langchain bitsandbytes
%pip install sentence_transformers
%pip install llama_index
%pip install -U langchain-community
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-langchain
%pip install chromadb
%pip install llama-index-vector-stores-chroma
%pip install PyMuPDF sentence-transformers

  Using cached pypdf-5.0.1-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-5.0.1-py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

  Using cached scikit_learn-1.5.2-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pillow-11.0.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached pillow-11.0.0-cp311-cp311-win_amd64.whl (2.6 MB)
Using cached scikit_learn-1.5.2-cp311-cp311-win_amd64.whl (11.0 MB)
Using cached scipy-1.14.1-cp311-cp311-win_amd64.whl (44.8 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Using cached llama_index-0.11.19-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_age

In [2]:
import os
import fitz  # PyMuPDF
import json

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.legacy.embeddings.langchain import LangchainEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient

c:\Users\Chitt\Downloads\LLAMA2_code\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Chitt\Downloads\LLAMA2_code\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\Chitt\Downloads\LLAMA2_code\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\Chitt\Downloads\LLAMA2_code\.venv\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model

In [3]:
with open('disease_description.json', 'r') as f:
    disease_description = json.load(f)  # Load the JSON data into a Python dictionary

In [4]:
def extract_text_from_pdf(file_path):
  with fitz.open(file_path) as pdf_document:
      text = ""
      for page in pdf_document:
          text += page.get_text()
      return text

In [5]:
def get_category_from_path(file_path):
    # Extract the folder name from the path to use it as the category
    folder_name = os.path.basename(os.path.dirname(file_path))
    return folder_name

In [6]:
# Create or load the persistent ChromaDB client
db = PersistentClient(path="./storage/chroma")

In [7]:
# Load the Sentence Transformer model for disease
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\Chitt\Downloads\LLAMA2_code\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chitt\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
def create_collection(name):
    # Initialize ChromaDB
    base_dir = 'data/data/' + name
    collection = db.get_or_create_collection(name)

    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith(".pdf"):
              file_path = os.path.join(root, file)

              # Extract text from the PDF
              text = extract_text_from_pdf(file_path)

              # Assign category based on folder structure
              category = get_category_from_path(file_path)

              # Generate embeddings for the document
              document_embedding = model.encode(text).tolist()

              # Store document with metadata in ChromaDB
              collection.add(
                  documents=[text],
                  embeddings=[document_embedding],
                  ids=[file_path],  # Use file path as a unique ID
                  metadatas=[{'category': category, "file_name": file}]
              )
    return collection

In [9]:
def delete_collection(collection_name):
    try:
        db.delete_collection(collection_name)
        print(f"Collection '{collection_name}' has been deleted successfully.")
    except Exception as e:
        print(f"An error occurred while deleting the collection: {e}")

# delete_collection("diet_plan")

In [10]:
disease_collection = create_collection("disease")

In [ ]:
# diet_plan_collection = create_collection("diet_plan")

In [11]:
# Querying the collection can be added here as needed
# For example, you can create a function to perform semantic searches
def query_chroma_db(collection, query_text, n_results=5):
    """Query the ChromaDB collection with a semantic search."""
    query_embedding = model.encode(query_text).tolist()

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    print(results)
    # for document, metadata in zip(results['documents'][0], results['metadatas'][0]):
    #     print(f"Filename: {metadata['filename']}\nContent: {document}\n")
    return results['metadatas']

In [12]:
# Example usage of querying the collection
user_query = "My age is 66years, height is 5.7ft and weight is 120 pounds. I have vomiting and loss of appetite. Suggest some diet plans."
response = query_chroma_db(disease_collection, user_query, n_results=1)
disease = response[0][0]['category']

{'ids': [[]], 'embeddings': None, 'documents': [[]], 'uris': None, 'data': None, 'metadatas': [[]], 'distances': [[]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


IndexError: list index out of range

In [13]:
!huggingface-cli login

^C


In [14]:
system_prompt = """
You are a nutrition AI assistant and your task is to suggest only diet plans. Below are the rules you need to follow:
- The input query will contain user disease.
- Consider disease and suggest diet plans to user using the context provided.
NO PREAMBLE.
"""
# - Your primary role is to suggest diet plans based on disease provided.
# - Provide clear, accurate, and relevant information to users, maintaining a friendly and professional tone.
# - Use context provided to find diet plans.
# - The input query will contain 2 paragraphs. First is about disease and second is about user specific details.
## Default format supportable by LLama2

# You are a nutrition AI assistant and your task is to suggest only diet plans. Below are the rules you need to follow:
# - The input query will contain 2 paragraphs. First paragraph tells you what disease user has and second paragraph says user age, height, weight and symptoms.
# - Consider disease and user age, height, weight and suggest diet plans to user using the context provided.
# - You need to understand the disease and user specific details to suggest appropriate diet plans.
# NO PREAMBLE.
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [15]:
import torch

# meta-llama/Meta-Llama-3.1-8B
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [ ]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

<ipython-input-23-4defcc445203>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=1024)

In [ ]:
# Load PDF documents from the diet_plan directory
diet_plan_documents = SimpleDirectoryReader("./data/data/diet_plan", recursive=True).load_data()

In [ ]:
# """ SAVE TO LOCAL"""
diet_plan_collection = db.get_or_create_collection("diet_plan")
vector_store = ChromaVectorStore(chroma_collection=diet_plan_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
diet_plan_index = VectorStoreIndex.from_documents(diet_plan_documents, storage_context=storage_context, show_progress=True)

Parsing nodes:   0%|          | 0/467 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/727 [00:00<?, ?it/s]

In [ ]:
# disease_documents = SimpleDirectoryReader("data/disease").load_data()
# # """ SAVE TO LOCAL"""
# db = chromadb.PersistentClient(path="./storage/chroma")
# chroma_collection = db.get_or_create_collection("disease")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# disease_index = VectorStoreIndex.from_documents(disease_documents, storage_context=storage_context, show_progress=True)

In [ ]:
# # fetch documents and index from storage
# db = chromadb.PersistentClient(path="./storage/chroma")
# chroma_collection = db.get_or_create_collection("diseases")
# chroma_vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# chroma_index = VectorStoreIndex.from_vector_store(vector_store=chroma_vector_store)

In [ ]:
print(f"Available collections: {db.list_collections()}")    # <- this returns the collection that I want to get

Available collections: [Collection(id=0b2deaec-bc7d-439b-9def-281fd954a280, name=diet_plan), Collection(id=f0053aab-a486-457c-b237-cce3b5e5d7e1, name=disease)]


In [ ]:
query_engine=diet_plan_index.as_query_engine()

In [ ]:
input_query = "{disease}".format(disease=disease_description[disease]) #  \n {user_query} , user_query=user_query
response=query_engine.query(input_query)
response

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Response(response="Thank you for reaching out! I'm here to help you with any questions or concerns you may have about diabetes. Based on the context information provided, it seems that you are looking for information on how to manage diabetes effectively.\n\nFirstly, it's important to understand that diabetes is a chronic health condition that requires ongoing management to prevent complications. The two primary types of diabetes are Type 1 and Type 2, and each has different causes and management strategies.\n\nType 1 diabetes is an autoimmune condition that destroys insulin-producing beta cells in the pancreas, requiring individuals to rely on insulin injections or an insulin pump for survival. Effective management of Type 1 diabetes involves monitoring blood sugar levels regularly, adjusting insulin doses accordingly, and maintaining a healthy diet and exercise routine.\n\nType 2 diabetes, on the other hand, is associated with insulin resistance and lifestyle factors such as obesity 

In [ ]:
response=query_engine.query("What is your name?")
response.response

In [ ]:
response=query_engine.query("How many times a week someone can eat fish?")
response.response

In [ ]:
response=query_engine.query("How are you trained?")
response.response